In [4]:
from tsl.ops.similarities import geographical_distance
import pandas as pd
import csv
import os 
from tqdm import tqdm

In [5]:
df = pd.read_csv('data\\aqs_sites.csv')


,State Code,County Code,Site Number,Latitude,Longitude,Datum,Elevation,Land Use,Location Setting,Site Established Date,...,Owning Agency,Local Site Name,Address,Zip Code,State Name,County Name,City Name,CBSA Name,Tribe Name,Extraction Date
0,01,1,1,32.437458,-86.472891,WGS84,64.0,RESIDENTIAL,SUBURBAN,1974-05-01,...,Al Dept Of Env Mgt,NaN,"KING ARTHUR TRAILER COURT, PRATTVILLE,AL",36067.0,Alabama,Autauga,Prattville,"Montgomery, AL",NaN,2021-11-24
1,01,1,2,32.428470,-86.443585,WGS84,0.0,AGRICULTURAL,RURAL,1980-01-01,...,Al Dept Of Env Mgt,NaN,COUNTY RD 4 PRATTVILLE EXPERIMENT ST,NaN,Alabama,Autauga,Prattville,"Montgomery, AL",NaN,2021-11-24
2,01,1,3,32.332659,-86.791521,WGS84,41.0,FOREST,RURAL,1989-08-31,...,Al Dept Of Env Mgt,NaN,"1170 COUNTY RD.15 SO., SELMA, AL. 36701",36003.0,Alabama,Autauga,Not in a City,"Montgomery, AL",NaN,2021-11-24
3,01,3,1,0.000000,0.000000,NAD27,0.0,UNKNOWN,RURAL,1959-01-01,...,US EPA National Exposure Research Lab,NaN,CHANDLER RESIDENCE FT MORGAN STATE PARK,NaN,Alabama,Baldwin,Not in a city,"Daphne-Fairhope-Foley, AL",NaN,2021-11-24
4,01,3,2,30.552367,-87.706911,WGS84,0.0,COMMERCIAL,RURAL,1974-01-01,...,Al Dept Of Env Mgt,NaN,COR PENNSYLVANIA & CHICAGO (ROBERTSDALE),36567.0,Alabama,Baldwin,Robertsdale,"Daphne-Fairhope-Foley, AL",NaN,2021-11-24


In [6]:
df = df.loc[:, "State Code":"Longitude"]

filename = "data\\aqs_sites_reduced.csv"

if not os.path.exists(filename):
    df.to_csv(filename, index = False)
else: print("File already present, skipped " + filename)

In [7]:
dist = geographical_distance(df.loc[:, ["Latitude","Longitude"]])
codes = df.loc[:, "State Code":"Site Number"]

,0,1,2,3,4,5,6,7,8,9,...,20747,20748,20749,20750,20751,20752,20753,20754,20755,20756
0,0.000000,2.926346,32.108792,9676.611868,240.051012,240.058687,253.665161,139.795039,118.588364,88.059774,...,2310.124300,2309.426022,2308.960437,2310.184694,2177.921452,2647.121383,2177.309158,1726.737896,2314.719617,2308.617148
1,2.926346,0.000000,34.366083,9673.829833,240.552921,240.569322,254.297445,136.882108,115.971529,90.871639,...,2312.930906,2312.230948,2311.767246,2312.990149,2180.720029,2649.981806,2176.074817,1726.100163,2317.017650,2310.911252
2,32.108792,34.366083,0.000000,9706.149445,216.166732,216.106015,228.722436,163.533484,134.228065,75.807803,...,2280.990872,2280.265217,2279.830372,2281.032464,2148.662052,2619.072555,2206.710232,1752.745285,2309.036108,2303.002818
3,9676.611868,9673.829833,9706.149445,0.000000,9787.992357,9788.639656,9804.473694,9548.424661,9591.827594,9749.272456,...,11985.798619,11985.230494,11984.618806,11985.949133,11854.149380,12309.295096,8216.813298,8812.137515,10955.000844,10947.577791
4,240.051012,240.552921,216.166732,9787.992357,0.000000,0.727555,17.690161,286.283913,235.123613,265.306746,...,2215.003506,2213.854914,2213.900648,2214.752293,2081.348177,2567.411732,2409.500989,1966.479135,2438.844779,2433.199816


In [5]:
filename = "data\dist_matrix.csv"

if not os.path.exists(filename):
    dist.to_csv(filename)
else: print("File already present, skipped " + filename)

In [29]:
codes.head()

,State Code,County Code,Site Number
0,01,1,1
1,01,1,2
2,01,1,3
3,01,3,1
4,01,3,2


In [8]:
def generate_id(site):
    state_code, county_code, site_num = site
    return f'{state_code}-{county_code}-{site_num}'

In [9]:
codes_id = pd.DataFrame(columns=["Site ID"])

for index, row in codes.iterrows():
   

   codes_id = pd.concat([codes_id, pd.DataFrame({"Site ID" : [generate_id((codes.loc[index, "State Code"],
                                 codes.loc[index, "County Code"],
                                 codes.loc[index, "Site Number"]))]})], ignore_index= True, axis=0)




,Site ID
0,01-1-1
1,01-1-2
2,01-1-3
3,01-3-1
4,01-3-2


In [18]:
with open('data\sites_dist.csv', mode = "w") as distfile:
    t_writer = csv.writer(distfile, delimiter=',')
    t_writer.writerow(["Site1", "Site2", "Dist"])

with open('data\sites_dist.csv', mode = "a", newline='') as distfile:
    t_writer = csv.writer(distfile, delimiter=',')

    for row in tqdm(range(dist.shape[0])):
        site1 = codes_id.loc[row,["Site ID"]].item()
        for col in tqdm(range(dist.shape[1]), leave = False):
            site2 = codes_id.loc[col,["Site ID"]].item()
            t_writer.writerow([site1, site2, dist.loc[row,col]])
        


  0%|          | 3/20757 [00:33<64:26:27, 11.18s/it]/s]


KeyboardInterrupt: 